In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
!pip install -q openai #--progress-bar off

In [3]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [4]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [5]:
output_datapath = "data/output/2023.11.21-5_Llama2_13B_API_5-few-shot/"

In [6]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [7]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [8]:
if USE_SAME_FEW_SHOTS==True:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
    data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
        (data_sel_4_few_shot_examples.text_id==5614) |
        (data_sel_4_few_shot_examples.text_id==2089) |
        (data_sel_4_few_shot_examples.text_id==5665) |
        (data_sel_4_few_shot_examples.text_id==370)  |
        (data_sel_4_few_shot_examples.text_id==1422) #|
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
    ]
else:
    data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

data_sel_4_few_shot_examples

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
250,370,10.1787/9789264272392-3-en,"For example, the poorest 20% of farm household...",2,0.0,9.0,1.000000
986,1422,10.1787/9789264283299-en,"Nevertheless, the average number of physician ...",3,1.0,8.0,0.777778
1456,2089,10.18356/ee52a573-en,"This refers to the manufacture, use and dispos...",12,3.0,29.0,0.812500
3936,5614,10.2139/SSRN.1406042,The modern definitions of the inchoate offence...,16,0.0,6.0,1.000000
3974,5665,10.18356/3fe10a08-en,"As a result, both the trade and current accoun...",8,0.0,5.0,1.000000


In [9]:
few_shots = ""

for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
    input_text = "Input text: \""+row['text']+"\"\n"
    output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
    few_shots += input_text
    few_shots += output_text
    
print(few_shots)

Input text: "For example, the poorest 20% of farm households in the Philippines tend to be small-scale producers and landless workers whose rice production does not satisfy their own needs and who therefore purchase 77% of the rice that they consume on average (Chapter 4). Regional levels of stunting among children under five years of age, for example, remain worryingly high (Figure 3). While economic growth has enabled significant declines in populations suffering from chronic poverty, for some households, it may have only altered the nature of food insecurity from permanent to transitory, rather than eliminate it entirely."
Output: "---SDG 2---"

Input text: "Nevertheless, the average number of physician contacts per person is not greater than the EU average. For example, home visits to patients by GPs are a regular practice and patients do not usually have to wait long to get access to GPs, although waiting times for more specialised services (e.g. mental health specialists, ophthal

In [10]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [11]:
template_part1 = """[INST]
<<SYS>>
In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

The following are 5 examples of expected classification given an input text:

"""+few_shots+"""

The text to be processed is:
<</SYS>>

'''
"""

template_part2 = """
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""

In [12]:
data.iloc[181:183]

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
181,571,10.14217/9781848591448-10-en,Three states had child mortality rates of five...,3,0.0,3.0,1.000000
182,4948,10.1787/9789264283473-en,Patients need a referral to access specialist ...,3,1.0,8.0,0.777778


In [13]:
# data_sel = data #.sample(2)
data_sel = data #.iloc[181:] #.sample(2)

In [14]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [15]:
import time

cnt = 0
# cnt = 181

results = []
for index, row in data_sel.iterrows():
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]
    
    query = template_part1 + text + template_part2
    
    body = {
      "model": "meta-llama/Llama-2-13b-chat-hf",
      "messages": [
          {"role": "system", "content": "You are a helpful assistant."}, 
          {"role": "user", "content": query}],
      "temperature": 0.0
    }
    
        
    try:
        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    except Exception as e:
        print("#############################################################################")
        print(e)
        print("#############################################################################")
        
    
    result_extended = (cnt, text_id, sdg_true, result)
    
    with open(output_datapath+f'sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    cnt+=1
    time.sleep(0.5)

0 6668 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 11---

The text is about a subway train derailing in Brooklyn, which is a transportation-related issue. SDG 11 focuses on sustainable cities and communities, and this incident is relevant to the goal's target of providing safe and accessible transportation systems.
1 6099 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 12---

The text is about Brooklyn Decker's photo shoot in the Maldives, which is a luxury vacation destination. The focus is on the beauty of the location and the model, rather than on any sustainable development issues. Therefore, the most appropriate SDG label is SDG 12, which focuses on responsible consumption and production.
2 6471 0   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
3 1786 0

24 1565 0   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 12---
25 1033 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text does not mention any specific SDGs directly, but it does mention the EU and NATO boosting support to Ukraine, which is related to SDG 16 (Peace, Justice and Strong Institutions).
26 5653 0   Sure! I'll do my best to classify the text according to the Sustainable Development Goals (SDGs) adopted by the United Nations Member States in 2015.

---SDG 8---

The text mentions "economic growth" and "commercial spaceports," which are related to SDG 8: Decent Work and Economic Growth.
27 940 0   Sure, I'd be happy to help! Here's the classification of the text based on the Sustainable Development Goals (SDGs):

---SDG 12---

The text discusses the weakness of the Chinese yuan and the potential for a "circuit breaker" in the currency ma

47 1427 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 15---

The text discusses the importance of species diversity and the potential key to preserving it, which aligns with SDG 15: Life On Land.
48 2023 0   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 16---

The text does not mention any specific SDGs directly, but it does mention "revolt in Russia" and "brace for what comes next," which suggests a focus on peace, justice, and strong institutions (SDG 16).
49 854 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 12---

The text does not mention any specific SDGs, but it does mention "inflation" and "consumer confidence," which are related to SDG 12 (Responsible Consumption and Production).
50 2541 0   Sure, I'd 

68 1658 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 8---
69 6667 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 1---

The text mentions "extreme poverty" and "child poverty," which are key indicators of SDG 1, "No Poverty." The text also highlights the high percentage of children living in poverty in certain countries, which is a concern for achieving this goal.
70 4588 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 2---

The text mentions "rural poor" and "smallholder producers," indicating a focus on poverty reduction and food security, which are both key aspects of SDG 2: Zero Hunger.
71 4057 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development G

93 5241 1   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 1---

The text discusses economic growth and poverty rates in Latin America, specifically mentioning the region's lowest poverty rate in the last 20 years and the highest per capita growth rates in Argentina and Panama. This information is relevant to SDG 1, which aims to end poverty in all its forms everywhere.
94 2724 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 1---

The text mentions the high child poverty rate in Mexico, which is a key indicator of poverty and inequality. The text also highlights the negative effects of child poverty on children's well-being and future prospects, which is a concern for SDG 1.
95 4303 1   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---

...

887 5734 14   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 14---
888 1089 14   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 14---

The text discusses the sustainable development of European aquaculture, which is related to SDG 14 (Life Below Water).
889 5290 14   Sure, I'd be happy to help! Here's the classification of the text you provided based on the Sustainable Development Goals (SDGs):

---SDG 14---

The text mentions the healthy fish stocks and the recreational fishery in Oresund, which is shared between Sweden and Denmark. This relates to SDG 14, which is to conserve and sustainably use the oceans, seas, and marine resources.
890 5720 14   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 14---

The text discusses the need for a m

920 1767 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 11---

The text establishes land-use planning and zoning principles to promote development that simultaneously protects and conserves the environment, which falls under SDG 11: Sustainable Cities and Communities.
921 1957 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 13---

The text mentions "altered ecosystems and habitats," "weather extremes and natural disasters," and "numerous risks to society" due to climate change, which falls under SDG 13: Climate Action.
922 980 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 15---
923 1299 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):



957 3934 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 15---
958 524 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 15---
959 4504 15   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 15---
960 6558 16   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the use of private military and security contractors (PMSCs) in peacekeeping operations, and the challenges in holding them accountable for human rights abuses. This is related to SDG 16, which focuses on promoting peaceful and inclusive societies, access to justice, and effective and accountable institutions.
961 4896 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
962 5813 16   Su

985 5742 16   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the issue of corruption and fraud in public investment contracts, which can lead to big losses incurred by the state budget. This is related to SDG 16, which aims to promote peaceful and inclusive societies, access to justice for all, and effective and accountable institutions.
986 1384 16   Sure, I'd be happy to help! Here's the label for the text you provided:

---SDG 16---
987 2092 16   Sure, I'd be happy to help! Here's the classification of the text based on the Sustainable Development Goals (SDGs):

---SDG 5---
988 2472 16   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the role of civil society groups in pressuring shareholders to divest from companies that cause ecological degradation or social inju

1007 1794 16   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 5---
1008 1864 16   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the individual's status in international law and their rights in criminal proceedings, specifically in the context of international criminal law and human rights law. This is related to SDG 16, which focuses on promoting peaceful and inclusive societies, access to justice for all, and effective and accountable institutions.
1009 973 16   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):

---SDG 16---

The text discusses the issue of revocation of citizenship, which is related to the theme of peace, justice, and strong institutions (SDG 16).
1010 535 16   Sure, I'd be happy to help! Here's th

In [16]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [17]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(865, 2639, 14, "  Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):\n\n---SDG 12---\n\nThe text mentions the importance of considering social and economic sustainability in fisheries management practices, particularly in developing countries. This relates to SDG 12, which focuses on responsible consumption and production patterns.")
(531, 6741, 8, "  Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs):\n\n---SDG 8---\n\nThe text discusses labor law enforcement and the dismissal of workers in India, which relates to SDG 8: Decent Work and Economic Growth.")
(256, 1972, 4, '  Sure, I\'d be happy to help! Here\'s the classification of the given text according to the Sustainable Development Goals (SDGs):\n\n---SDG 4---\n\nThe text mentions "school characteristics such as location and school size" and "socio-economically disadvantaged studen

In [18]:
len(tuple_list)

1020

In [19]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [20]:
results_processed

,cnt,id,true_label,predicted_label
0,865,2639,14,"Sure, I'd be happy to help! Here's the class..."
1,531,6741,8,"Sure, I'd be happy to help! Here's the class..."
2,256,1972,4,"Sure, I'd be happy to help! Here's the class..."
3,816,4186,13,"Sure, I'd be happy to help! Here's the class..."
4,444,2876,7,"Sure, I'd be happy to help! Here's the class..."
...,...,...,...,...
1015,487,433,8,"Sure, I'd be happy to help! Here's the class..."
1016,34,6228,0,"Sure! Here's the text you provided, with my ..."
1017,37,4226,0,"Sure! Here's the text you provided, with the..."
1018,907,3261,15,"Sure, I'd be happy to help! Here's the class..."


In [21]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the class..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,"Sure, I'd be happy to help! Here's the label..."
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label..."
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the class..."
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the class..."
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the class..."
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the class..."


In [22]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class...",11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the class...",12
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,"Sure, I'd be happy to help! Here's the label...",16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label...",4
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the class...",16
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Here's the class...",16
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the class...",16
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the class...",16


In [23]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
1,73,73,73,73,73,73,73,73,73,73,73
10,25,25,25,25,25,25,25,25,25,25,25
11,52,48,52,52,48,48,48,52,52,52,52
12,84,70,84,84,70,70,70,84,84,84,84
13,54,53,54,54,53,53,53,54,54,54,54
14,40,40,40,40,40,40,40,40,40,40,40
15,68,66,68,68,66,66,66,68,68,68,68
16,78,69,78,78,69,69,69,78,78,78,78
17,18,18,18,18,18,18,18,18,18,18,18


In [24]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_13B.xlsx')

In [25]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        60
           1       0.62      0.75      0.68        60
           2       0.92      0.73      0.81        60
           3       0.70      0.97      0.81        60
           4       0.56      0.98      0.71        60
           5       0.85      0.75      0.80        60
           6       0.98      0.87      0.92        60
           7       0.89      0.67      0.76        60
           8       0.41      0.55      0.47        60
           9       0.63      0.63      0.63        60
          10       0.80      0.33      0.47        60
          11       0.71      0.62      0.66        60
          12       0.56      0.78      0.65        60
          13       0.80      0.72      0.75        60
          14       0.95      0.63      0.76        60
          15       0.79      0.90      0.84        60
          16       0.64      0.83      0.72        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.